In [1]:
import numpy as np
import pandas as pd
import comonotonic as cm
import os
from sklearn.model_selection import train_test_split
import copy
import utils
import matplotlib.pyplot as plt

In [2]:
# Note that we assume the last column is the label
# If the first column is ID's, remove it
filename = "pulsar_stars.csv"
df = pd.read_csv("Datasets/"+filename)
colnames = [('X'+str(i)) for i in range(df.shape[1]-1)]
colnames.append('Y')
df.columns = colnames
df

,X0,X1,X2,X3,X4,X5,X6,X7,Y
0,140.562500,55.683782,-0.234571,-0.699648,3.199833,19.110426,7.975532,74.242225,0
1,102.507812,58.882430,0.465318,-0.515088,1.677258,14.860146,10.576487,127.393580,0
2,103.015625,39.341649,0.323328,1.051164,3.121237,21.744669,7.735822,63.171909,0
3,136.750000,57.178449,-0.068415,-0.636238,3.642977,20.959280,6.896499,53.593661,0
4,88.726562,40.672225,0.600866,1.123492,1.178930,11.468720,14.269573,252.567306,0
...,...,...,...,...,...,...,...,...,...
17893,136.429688,59.847421,-0.187846,-0.738123,1.296823,12.166062,15.450260,285.931022,0
17894,122.554688,49.485605,0.127978,0.323061,16.409699,44.626893,2.945244,8.297092,0
17895,119.335938,59.935939,0.159363,-0.743025,21.430602,58.872000,2.499517,4.595173,0
17896,114.507812,53.902400,0.201161,-0.024789,1.946488,13.381731,10.007967,134.238910,0


In [10]:
# use the original data for Naive Bayes
original_X = df[colnames[:-1]].to_numpy()
original_Y = df[colnames[-1]].to_numpy()
original_X_train, original_X_test, original_Y_train, original_Y_test = train_test_split(original_X, original_Y, test_size=0.2, random_state=42)


df_copy = df.copy()
num_categories = 5
for col_idx in [i for i in range(8)]:
    discretized_col = pd.cut(df_copy.iloc[:,col_idx],num_categories, labels=[i for i in range(num_categories)])
    df_copy['X'+str(col_idx)] = discretized_col
# For pure comonotonic classifier
X = df_copy[colnames[:-1]].to_numpy()
Y = df_copy[colnames[-1]].to_numpy()
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [11]:
pure_como_classifier = cm.pure_comonotonic(X_train, Y_train, None)
pure_como_classifier.run()
Y_predict = pure_como_classifier.predict(X_test)
accuracy = utils.get_accuracy(Y_predict, Y_test)
accuracy

0.9103351955307263

In [5]:
# comparation with Naive Bayes
from sklearn.naive_bayes import GaussianNB
model = GaussianNB()
model.fit(original_X_train, original_Y_train)
pred = model.predict(original_X_test)
utils.get_accuracy(pred, original_Y_test)

0.9474860335195531